### 4-ApplyImageScale.ipynb

Applies the image specific mm to pixel scale for each live bee image.

In [1]:
from pathlib import Path
import pandas as pd
import cv2
from tqdm import tqdm

In [2]:
# Scale options
MARKER_LENGTH_MM = 5.0
PIXELS_PER_MM = 150

# Define directories
data_dir = Path("/mnt/c/Projects/Master/Data/")
csv_path = data_dir / "Processed" / "AgePrediction" / "01-Scaled" / "MarkerLenghts.csv"
images_folder = data_dir / "Processed" / "AgePrediction" / "01-Scaled" / "1-LiveBeeWings"
output_folder = data_dir / "Processed" / "AgePrediction" / "01-Scaled" / "2-LiveBeeScaled"

DEBUG = False

In [3]:
# Make sure output directory exists
output_folder.mkdir(parents=True, exist_ok=True)

# Load and process CSV
df = pd.read_csv(csv_path)
df["Filename"] = df["Filename"].str.replace(".JPG", ".png", case=False)

# Report missing values
count_nan = df["MarkerLengthInPixels"].isnull().sum()
print(f"Number of NaN values present: {count_nan}")

# Create lookup dictionary
marker_lengths = dict(zip(df["Filename"], df["MarkerLengthInPixels"]))

# Process images
for image_path in tqdm(sorted(images_folder.glob("*.png")), desc="Processing files", ncols=145):
    filename = image_path.name

    if filename not in marker_lengths:
        print(f"Marker length missing for {filename}, skipping.")
        continue

    marker_length_px = marker_lengths[filename]
    if pd.isna(marker_length_px):
        print(f"Marker length is NaN for {filename}, skipping.")
        continue

    # Compute scale factor and new size
    current_pixels_per_mm = marker_length_px / MARKER_LENGTH_MM
    scale_factor = PIXELS_PER_MM / current_pixels_per_mm

    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Could not read {filename}, skipping.")
        continue

    new_size = (
        int(image.shape[1] * scale_factor),
        int(image.shape[0] * scale_factor),
    )
    resized_image = cv2.resize(image, new_size, interpolation=cv2.INTER_LINEAR)

    output_path = output_folder / filename
    cv2.imwrite(str(output_path), resized_image)

    if DEBUG:
        print(f"Scaled {filename}: original marker {marker_length_px:.2f}px, scale factor {scale_factor:.4f}")

Number of NaN values present: 0


Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████████| 999/999 [02:04<00:00,  8.04it/s]
